In [1]:
import xml.etree.ElementTree as ET
import numpy as np
import cv2, json
import shutil
from PIL import Image
import os
from boundingbox_utils import *

In [7]:
def annotations_yolo(dataset_dir , image_to_box, image_gt, pixel_array,
                     categories_to_colors, ordered_categories):
    
    image_dir= dataset_dir+ '/images/'
    annotations_dir= dataset_dir+ '/annotations/'
    
    txt_file = image_gt.split('/')[-1].replace('png', 'txt')
    txt_file = annotations_dir + txt_file
    
    img_file = image_to_box.split('/')[-1]
    img_file = image_dir + img_file
    print(img_file)
    
    if not os.path.exists(dataset_dir):
        os.mkdir(dataset_dir)
    if not os.path.exists(image_dir):
        os.mkdir(image_dir)
    if not os.path.exists(annotations_dir):
        os.mkdir(annotations_dir)
    
    boxes_list = GetBoxes(image_gt)
    category_map = GetCategoriesJson(categories_to_colors)
    box_map = GetBoxesJson(boxes_list, categories_to_colors, ordered_categories, pixel_array)
    image_map = GetImageJson(image_to_box)
    lines = []
    if len(box_map)==1:
        for box in box_map:
            if box['category'] == 'default':
                return
    if len(box_map)==0:
        return
    
    for box in box_map:
        print(box)
        if box['category'] == 'default':
            continue
        else:
            # (x,y) is the top-left coordinate of the rectangle and (w,h) are its width and height.
            # to change this to YOLO format, we need to normalize
            imageh = 1080
            imagew = 1920
            # modify x value
            box['bbox'][0] = round((box['bbox'][0] + box['bbox'][2]/2)/ imagew, 15)
            box['bbox'][1] = round((box['bbox'][1] + box['bbox'][3]/2)/ imageh, 15)
            box['bbox'][2] = round((box['bbox'][2])/ imagew, 10)
            box['bbox'][3] = round((box['bbox'][3])/ imageh, 10)
            
            line = str(category_map[box['category']]) + ' '
            box['bbox'] = map(str, box['bbox']) 
            line = line + ' '.join(box['bbox'])
            lines.append(line)
    lines = '\n'.join(lines)
    with open(txt_file, "w") as f:
        f.write(lines)
        f.close()
    cv2.imwrite(img_file, cv2.imread(image_to_box))

In [8]:
palette_image = "color_pallete.tga"
xml_file = "GroundTruthColorMapping.xml"
pixel_array = GetColorMap(palette_image)
categories_to_colors, ordered_categories = MapColorPalette(xml_file)

dataset_dir = './annotated_images'
image_to_box = "sample_images/5.png"
image_gt = "sample_images/5_mask.png"

annotations_yolo(dataset_dir, image_to_box, image_gt, pixel_array,
                     categories_to_colors, ordered_categories)

./annotated_images/images/5.png
{'category': 'default', 'bbox': [0, 0, 1920, 1080], 'color': (50, 239, 13), 'instance_id': 77, 'num_pixels': 1576992}
{'category': 'Boat', 'bbox': [512, 153, 1116, 614], 'color': (55, 3, 93), 'instance_id': 20, 'num_pixels': 173989}
{'category': 'Boat', 'bbox': [359, 457, 169, 140], 'color': (86, 134, 205), 'instance_id': 65, 'num_pixels': 12462}
